# Overview

At a very high level, Howso Engine is about: 

- Making an accurate prediction (even with limited or sparse data!) 

- Explaining the prediction process 

- Showing key properties of the data 

In this notebook, we will be using the adult data set as an example to demonstrate some of Howso Engine’s capabilities, including cases and features which contribute to predictions, anomalies analysis, and potential improvements to the data to gain more insight into the data.  


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_feature_importances, plot_anomalies, plot_dataset

In [2]:
# Load adult data
df = pd.read_csv('data/adult.data', header=None)

# Specify column names
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain', 
              'capital-loss', 'hours-per-week', 'native-country', 'target']

# Sample the data for demo purpose
df = df.sample(1_000).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,34,Local-gov,284843,HS-grad,9,Never-married,Farming-fishing,Not-in-family,Black,Male,594,0,60,United-States,<=50K
1,38,Private,64879,Some-college,10,Divorced,Sales,Not-in-family,White,Female,0,0,40,United-States,<=50K
2,35,Private,360799,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,40,United-States,>50K
3,22,Private,359759,Some-college,10,Never-married,Sales,Not-in-family,Asian-Pac-Islander,Male,0,0,20,Philippines,<=50K
4,20,?,133515,Some-college,10,Never-married,?,Own-child,White,Female,0,0,15,France,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,42,Private,248356,HS-grad,9,Never-married,Sales,Unmarried,White,Female,0,0,40,United-States,<=50K
996,28,Private,274679,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States,<=50K
997,63,Private,158199,1st-4th,2,Widowed,Machine-op-inspct,Unmarried,White,Female,0,0,44,Portugal,<=50K
998,41,?,128700,HS-grad,9,Never-married,?,Own-child,White,Female,0,0,40,United-States,<=50K


In [3]:
partial_features = {
    "education": {"type": "nominal"}
}

# Infer features types
features = infer_feature_attributes(df, features=partial_features)

# Specify the context and action feature
action_features = ['target']
context_features = features.get_names(without=['target'])

In [4]:
# Create the trainee with custom name
t = engine.Trainee(name='Engine - Predictions and Explanations Recipe', features=features, overwrite_existing=True)

# Train
t.train(df)

# Analyze the model
t.analyze(action_features=action_features)


In [5]:
t.react_into_trainee(residuals=True)

accuracy = t.get_prediction_stats(stats=['accuracy'])['target'][0]

print("Test set prediction accuracy: {acc}".format(acc=accuracy))

Test set prediction accuracy: 0.821


# Explain

How was the predictions made? 

Howso Engine provides detailed explanation for complete model transparency. Let's examine a subset of the explanations.


## Feature importance (global)

The feature importance information provides insight into the feature[s] which were primary drivers for each of the prediction. This is important to understand in the context of AI bias and discrimination (ex. Sensitive attribute being the primary contribution to a prediction). 

This information is available at the global level (overall model), but can also be extracted at the local level (regional model for each case).

In [6]:
# Extract the global MDA (mean decrease in accuracy)
t.react_into_trainee(action_feature=action_features[0], mda_robust=True, residuals=True)
global_mda = t.get_prediction_stats(action_feature=action_features[0], stats=['mda'])
plot_feature_importances(global_mda, title="Global Mean Decrease in Accuracy (MDA)", yaxis_title="MDA")

## Feature uncertainty (global)

Are there any noisy features? 

Howso Engine’s performance is robust against noisy feature[s], and can maintain a high level of accuracy despite noisy data. 

Part of the reason  Howso Engine can maintain the level of performance despite noisy data is through characterization of feature uncertainties (residuals). The feature residuals can be extracted for user review. Note, the residuals are in the same units as the original features which makes it easy to interpret. For example, the residual for the “age” feature has the unit of years as in the original data.

Feature residuals are available at the global level (overall model) and at the local level (regional model for each case).


In [7]:
# Global feature residuals
global_feature_residuals = t.get_prediction_stats(stats=['mae']).T.rename(columns={'mae':'residuals'}).sort_values('residuals', ascending=False)
global_feature_residuals.iloc[0:10]

,residuals
fnlwgt,96858.530174
capital-gain,1517.384594
hours-per-week,10.316415
age,9.699608
occupation,0.772539
workclass,0.395709
relationship,0.370735
marital-status,0.320135
education-num,0.280954
education,0.280954


# "Show me..."

Howso Engine can be used to show interesting information pertaining to the data and model, such as anomalous cases and potential model improvements. 
 
For each prediction, Howso Engine can also extract the influential cases and boundary cases to provide an exact explanation to the prediction process. More details on what’s available can be found in the notebook “2-interpretability.ipynb”.


## Anomalous cases

Anomalous cases can exist in the data as either an outlier or inlier. Outliers are cases which are very different than other cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 



In [8]:
# Store the familiarity conviction, this will be used to identify anomalous cases
t.analyze()
t.react_into_features(familiarity_conviction_addition=True, distance_contribution=True)
stored_convictions = t.get_cases(session=t.active_session, features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution'])

stored_convictions

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution
0,34,Local-gov,284843,HS-grad,9,Never-married,Farming-fishing,Not-in-family,Black,Male,594,0,60,United-States,<=50K,21.938997,0,7bbda081-cd7c-46fd-9911-87746c6ac9a8,3.975991
1,38,Private,64879,Some-college,10,Divorced,Sales,Not-in-family,White,Female,0,0,40,United-States,<=50K,1.561928,1,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.213805
2,35,Private,360799,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,40,United-States,>50K,8.757461,2,7bbda081-cd7c-46fd-9911-87746c6ac9a8,3.316372
3,22,Private,359759,Some-college,10,Never-married,Sales,Not-in-family,Asian-Pac-Islander,Male,0,0,20,Philippines,<=50K,1.002890,3,7bbda081-cd7c-46fd-9911-87746c6ac9a8,5.943205
4,20,?,133515,Some-college,10,Never-married,?,Own-child,White,Female,0,0,15,France,<=50K,1.088377,4,7bbda081-cd7c-46fd-9911-87746c6ac9a8,5.879643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,42,Private,248356,HS-grad,9,Never-married,Sales,Unmarried,White,Female,0,0,40,United-States,<=50K,4.223326,995,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.685854
996,28,Private,274679,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States,<=50K,3.753311,996,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.935222
997,63,Private,158199,1st-4th,2,Widowed,Machine-op-inspct,Unmarried,White,Female,0,0,44,Portugal,<=50K,0.949989,997,7bbda081-cd7c-46fd-9911-87746c6ac9a8,6.065232
998,41,?,128700,HS-grad,9,Never-married,?,Own-child,White,Female,0,0,40,United-States,<=50K,38.792509,998,7bbda081-cd7c-46fd-9911-87746c6ac9a8,3.271994


In [9]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.75

# Extract the anomalous cases
low_convicts = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold ].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = ['inlier' if d < average_dist_contribution else 'outlier' for d in low_convicts['distance_contribution']]
low_convicts['category'] = cat

## Outliers

Let’s examine a few outlier cases. Outliers are cases which are very different than other cases.

In [10]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,39,Private,190297,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,55,United-States,>50K,0.017773,960,7bbda081-cd7c-46fd-9911-87746c6ac9a8,25.399846,outlier
1,55,Self-emp-inc,392325,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,99999,0,60,United-States,>50K,0.017786,678,7bbda081-cd7c-46fd-9911-87746c6ac9a8,25.407325,outlier
2,43,Private,143582,5th-6th,3,Married-civ-spouse,Machine-op-inspct,Wife,Asian-Pac-Islander,Female,0,2129,72,?,<=50K,0.143289,827,7bbda081-cd7c-46fd-9911-87746c6ac9a8,11.834850,outlier
3,20,?,114746,11th,7,Married-spouse-absent,?,Own-child,Asian-Pac-Islander,Female,0,1762,40,South,<=50K,0.144538,629,7bbda081-cd7c-46fd-9911-87746c6ac9a8,11.785548,outlier
4,32,Federal-gov,148138,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,2002,40,Iran,<=50K,0.159850,584,7bbda081-cd7c-46fd-9911-87746c6ac9a8,11.237330,outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,66,Private,236879,Preschool,1,Widowed,Priv-house-serv,Other-relative,White,Female,0,0,40,Guatemala,<=50K,0.677209,490,7bbda081-cd7c-46fd-9911-87746c6ac9a8,6.615105,outlier
76,90,Private,90523,HS-grad,9,Widowed,Transport-moving,Unmarried,White,Male,0,0,99,United-States,<=50K,0.701678,624,7bbda081-cd7c-46fd-9911-87746c6ac9a8,6.559177,outlier
77,50,Private,123174,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Black,Male,0,0,37,?,>50K,0.701827,681,7bbda081-cd7c-46fd-9911-87746c6ac9a8,6.559653,outlier
78,66,Private,180211,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,30,Philippines,<=50K,0.717280,389,7bbda081-cd7c-46fd-9911-87746c6ac9a8,6.506798,outlier


In [11]:
# Cache global non-robust residuals into trainee
t.react_into_trainee(residuals=True)

# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {'robust_computation': True,
           'global_case_feature_residual_convictions': True, 
           'local_case_feature_residual_convictions': True}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=outliers_indices, 
                  preserve_feature_values=df.columns.tolist(), 
                  leave_case_out=True, 
                  details=details)

In [12]:
# Extract the global and local case feature residual convictions
global_case_feature_residual_convictions = pd.DataFrame(results['explanation']['global_case_feature_residual_convictions'])[df.columns.tolist()]
local_case_feature_residual_convictions = pd.DataFrame(results['explanation']['local_case_feature_residual_convictions'])[df.columns.tolist()]

In [13]:
plot_anomalies(outliers, local_case_feature_residual_convictions, title="Outliers", yaxis_title="Residual Conviction")

The heat map explains the reason why each case was an outlier. The darker the shade of red, the higher the contribution to the case being an outlier. 

## Inliers

Let’s examine a few inlier cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 

In [14]:
# Get the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,42,Private,213464,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.128645,873,7bbda081-cd7c-46fd-9911-87746c6ac9a8,0.477670,inlier
1,43,Private,212894,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.129565,663,7bbda081-cd7c-46fd-9911-87746c6ac9a8,0.480390,inlier
2,46,Private,106255,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.132972,318,7bbda081-cd7c-46fd-9911-87746c6ac9a8,0.534098,inlier
3,47,Private,106255,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.133871,847,7bbda081-cd7c-46fd-9911-87746c6ac9a8,0.536693,inlier
4,20,Private,203027,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,40,United-States,<=50K,0.141201,88,7bbda081-cd7c-46fd-9911-87746c6ac9a8,0.582256,inlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,32,Private,132601,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K,0.734703,223,7bbda081-cd7c-46fd-9911-87746c6ac9a8,1.753836,inlier
137,30,Private,185177,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,43,United-States,<=50K,0.743814,369,7bbda081-cd7c-46fd-9911-87746c6ac9a8,1.772794,inlier
138,24,Private,179423,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,<=50K,0.744830,309,7bbda081-cd7c-46fd-9911-87746c6ac9a8,1.746266,inlier
139,17,Private,108402,10th,6,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K,0.747913,781,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.132349,inlier


In [15]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=inliers_indices, 
                  preserve_feature_values=df.columns.tolist(), 
                  leave_case_out=True, 
                  details=details)

In [16]:
# Extract the global and local case feature residual convictions
global_case_feature_residual_convictions = pd.DataFrame(results['explanation']['global_case_feature_residual_convictions'])[df.columns.tolist()]
local_case_feature_residual_convictions = pd.DataFrame(results['explanation']['local_case_feature_residual_convictions'])[df.columns.tolist()]

In [17]:
plot_anomalies(inliers, local_case_feature_residual_convictions, title="Inliers", yaxis_title="Residual Conviction")

The heat map explains the reason why each case was an inlier. The darker the shade of blue, the higher the contribution the to case being an inlier.

## Potential improvements

Sparse regions of the model or under defined problems can make it difficult to make an accurate prediction. Howso Engine can be used to identify potential data, or model improvements by examining the residual conviction and density.

In [18]:
# Identify cases for investigation
partial_train_df = stored_convictions
partial_train_cases = partial_train_df[['.session', '.session_training_index']]


In [19]:
# Residual convictions are output via the local_case_feature_residual_convictions explanation
details = {'global_case_feature_residual_convictions':True}

# Get the residual convictions for the specified cases
new_result = t.react(case_indices=partial_train_cases.values.tolist(), 
                     leave_case_out=True, 
                     preserve_feature_values=df.drop(action_features, axis=1).columns.tolist(), 
                     action_features=action_features,
                     details=details)

In [20]:
# Extract residual conviction
target_residual_convictions = [ x['target'] for x in new_result['explanation']['global_case_feature_residual_convictions'] ]

# Binarize residual conviction
convict_threshold = 0.75
low_residual_conviction = [1 if x <= convict_threshold else 0 for x in target_residual_convictions]

# Density is just the inverse of distance_contribution
density = 1 / partial_train_df['distance_contribution']

# Add new features to the dataframe
partial_train_df['density'] = density
partial_train_df['target_residual_conviction'] = target_residual_convictions
partial_train_df['low_residual_conviction'] = low_residual_conviction

In [21]:
partial_train_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,density,target_residual_conviction,low_residual_conviction
0,34,Local-gov,284843,HS-grad,9,Never-married,Farming-fishing,Not-in-family,Black,Male,...,60,United-States,<=50K,21.938997,0,7bbda081-cd7c-46fd-9911-87746c6ac9a8,3.975991,0.251510,1.370286e+00,0
1,38,Private,64879,Some-college,10,Divorced,Sales,Not-in-family,White,Female,...,40,United-States,<=50K,1.561928,1,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.213805,0.451711,2.142465e+02,0
2,35,Private,360799,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,>50K,8.757461,2,7bbda081-cd7c-46fd-9911-87746c6ac9a8,3.316372,0.301534,2.142465e+02,0
3,22,Private,359759,Some-college,10,Never-married,Sales,Not-in-family,Asian-Pac-Islander,Male,...,20,Philippines,<=50K,1.002890,3,7bbda081-cd7c-46fd-9911-87746c6ac9a8,5.943205,0.168259,2.142465e+02,0
4,20,?,133515,Some-college,10,Never-married,?,Own-child,White,Female,...,15,France,<=50K,1.088377,4,7bbda081-cd7c-46fd-9911-87746c6ac9a8,5.879643,0.170078,2.142465e+02,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,42,Private,248356,HS-grad,9,Never-married,Sales,Unmarried,White,Female,...,40,United-States,<=50K,4.223326,995,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.685854,0.372321,3.769717e+11,0
996,28,Private,274679,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Male,...,50,United-States,<=50K,3.753311,996,7bbda081-cd7c-46fd-9911-87746c6ac9a8,2.935222,0.340690,4.797237e-01,1
997,63,Private,158199,1st-4th,2,Widowed,Machine-op-inspct,Unmarried,White,Female,...,44,Portugal,<=50K,0.949989,997,7bbda081-cd7c-46fd-9911-87746c6ac9a8,6.065232,0.164874,2.142465e+02,0
998,41,?,128700,HS-grad,9,Never-married,?,Own-child,White,Female,...,40,United-States,<=50K,38.792509,998,7bbda081-cd7c-46fd-9911-87746c6ac9a8,3.271994,0.305624,1.983163e+11,0


In [22]:
# Helper function to resize the data points
def get_sizes(min_size, max_size, series):
    min_value = series.min()
    max_value = series.max()
    
    m = (max_size - min_size) / (max_value - min_value)
    
    sizes = series * m + min_size
    return (sizes)

partial_train_df["density"] = get_sizes(5, 500, partial_train_df["density"])

In [23]:
plot_dataset(partial_train_df, x="age", y="education-num", size="density", hue="low_residual_conviction", alpha=0.4)

The above graph is a visualization of the data set in 2-dimensions, with the color as an indication of residual conviction and the size representing the density of the data. More specifically, the orange color represents the low conviction points (points which are very uncertain), and small size represents low density. Therefore, adding more data to the region with small, orange points can improve model performance. 


On the other hand, an orange point that is large would be an indication that this case lies in an dense region but was not predictable. Hence, this will be an indication where the problem is not well defined, or the data is missing key features.  
